# CUDA を使って画像に3DLUTをかける

## 3DLUT の補間処理の元ネタ

FUJIFILM社の「LOGとLUTの基礎講座」資料 P.38 のアルゴリズムを使用する。

| |
|:-----:|
|![figure1](./figure/3dlut_interpolation.jpg)|


リンク：[LOGとLUTの基礎講座](http://fujifilm.jp/business/broadcastcinema/solution/color_management/is-mini/promotion/pack/pdf/news/20150122_jppa.pdf)

## Python向けに補間処理の数式を考える。

![figure1](./figure/sekkei.jpg)

8頂点で囲われた立方体を上図を参考に8領域に分ける。それぞれの領域の体積を$V_0 \sim V_7$ と置くと、以下の式で算出できる。

\begin{align}
V_0 & = R_0 \cdot G_0 \cdot B_0 \\
V_1 & = R_0 \cdot G_0 \cdot B_1 \\
V_2 & = R_0 \cdot G_1 \cdot B_0 \\
V_3 & = R_0 \cdot G_1 \cdot B_1 \\
V_4 & = R_1 \cdot G_0 \cdot B_0 \\
V_5 & = R_1 \cdot G_0 \cdot B_1 \\
V_6 & = R_1 \cdot G_1 \cdot B_0 \\
V_7 & = R_1 \cdot G_1 \cdot B_1 \\
\end{align}

ここで $V'_i = 1 - V_i$ と置くと、3DLUTの出力は以下の式で求められる。

$$
\begin{align}
R_{out} & = V'_0 \cdot L_0[R] + V'_1 \cdot L_1[R] + V'_2 \cdot L_2[R] + V'_3 \cdot L_3[R] \\
& + V'_4 \cdot L_4[R] + V'_5 \cdot L_5[R] + V'_6 \cdot L_6[R] + V'_7 \cdot L_7[R] \\
G_{out} & = V'_0 \cdot L_0[G] + V'_1 \cdot L_1[G] + V'_2 \cdot L_2[G] + V'_3 \cdot L_3[G] \\ 
& + V'_4 \cdot L_4[G] + V'_5 \cdot L_5[G] + V'_6 \cdot L_6[G] + V'_7 \cdot L_7[G] \\
B_{out} & = V'_0 \cdot L_0[B] + V'_1 \cdot L_1[B] + V'_2 \cdot L_2[B] + V'_3 \cdot L_3[B] \\
& + V'_4 \cdot L_4[B] + V'_5 \cdot L_5[B] + V'_6 \cdot L_6[B] + V'_7 \cdot L_7[B]
\end{align}
$$

## Grid数を考慮した数式を立ててみよう

グリッド数を$N$とする。$N$ は整数 $m$ によって以下の式で求められる。<br>
ひとまずは、m=4, 5, 6 あたりをターゲットとする。

$$
N = 2^m + 1
$$

入力画像を $D(w, h, c)$ とする。$w, h$ は空間方向の座標を $c$ は 色方向の座標を意味する。<br>
入力画像は$N$に合わせて正規化する。正規化後の入力画像を $D'$ とする。

$$
D' = \frac{D}{{\rm max}(D)} \cdot N
$$

さて、この時 $D'$ が参照するグリッドのインデックス $i, j, k$ は以下となる。<br>
※ $D=N$ つまり最大値の時に、本当に端っこのグリッドに当たるか不安。要確認。

$$
i = \lfloor D' \rfloor
$$

# 3DLUT の 読み込みと書き込み

## 使う形式
**.cube** 形式を使う。詳細は Adobe の [Cube LUT Specification](http://wwwimages.adobe.com/content/dam/Adobe/en/products/speedgrade/cc/pdfs/cube-lut-specification-1.0.pdf) 
参照。

## ザックリと .cube 形式について説明

### データの中身
エディタで .cube を開くと以下のようになってる。<br>
したの方に実際の 3DLUT データが詰まってる。<br>
データは左から順に Red/Green/Blue となっている。
```
# DaVinci Resolve Cube (3D LUT).
#
# Converts Rec.709 data to DCI-XYZ.
# Gamma as defined by ITU-Rec.1886.
# 

LUT_3D_SIZE 33
LUT_3D_INPUT_RANGE 0.0000000000 1.0000000000

0.0000000000 0.0000000000 0.0000000000
0.0280636951 0.0217522469 0.0086490514
0.0532131180 0.0412456335 0.0163999427
0.0773685559 0.0599685796 0.0238444942
0.1009003238 0.0782081173 0.0310968346
0.1239789517 0.0960964251 0.0382095201
0.1467027800 0.1137097266 0.0452128587
0.1691357380 0.1310975738 0.0521265529
0.1913226612 0.1482947187 0.0589644207
0.2132966958 0.1653268532 0.0657366777
0.2350833187 0.1822137243 0.0724511756
0.2567027062 0.1989709707 0.0791141326
0.2781712211 0.2156112755 0.0857305916
0.2995023928 0.2321451250 0.0923047223
(以下省略)
```

### 3DLUT のグリッドのインクリメントの順番
1行ごとに3DLUTの Index がインクリメントするが、その順序は Red, Green, Blue となっている。<br>
例えば、3x3x3 の 3DLUT だった場合、以下のように各色の Index が増加する。

| 行番号 | Red | Green | Blue | 
|-----|-----|-------|------| 
| 0   | 0   | 0     | 0    | 
| 1   | 1   | 0     | 0    | 
| 2   | 2   | 0     | 0    | 
| 3   | 0   | 1     | 0    | 
| 4   | 1   | 1     | 0    | 
| 5   | 2   | 1     | 0    | 
| 6   | 0   | 2     | 0    | 
| 7   | 1   | 2     | 0    | 
| 8   | 2   | 2     | 0    | 
| 9   | 0   | 0     | 1    | 
| 10  | 1   | 0     | 1    | 
| 11  | 2   | 0     | 1    | 
| 12  | 0   | 1     | 1    | 
| 13  | 1   | 1     | 1    | 
| 14  | 2   | 1     | 1    | 
| 15  | 0   | 2     | 1    | 
| 16  | 1   | 2     | 1    | 
| 17  | 2   | 2     | 1    | 
| 18  | 0   | 0     | 2    | 
| 19  | 1   | 0     | 2    | 
| 20  | 2   | 0     | 2    | 
| 21  | 0   | 1     | 2    | 
| 22  | 1   | 1     | 2    | 
| 23  | 2   | 1     | 2    | 
| 24  | 0   | 2     | 2    | 
| 25  | 1   | 2     | 2    | 
| 26  | 2   | 2     | 2    | 


In [40]:
# CUBE形式の3DLUTの読み書きモジュールを作る
# -----------------------------------------
import sys
import imp
sys.path.append('./code')
import pycuda_3dlut as p3
imp.reload(p3)
lut_data = p3.load_3dlut_cube('./data/Rec709 to DCI-XYZ.cube')
p3.save_3dlut_cube(lut_data, './data/save_test.cube')

lut_size = 33x33x33
data start line = 10


In [17]:
# 上のコードを関数化
# ---------------------------------
import sys
import imp
sys.path.append('./code')
import pycuda_3dlut as p3
imp.reload(p3)
p3.mono_conv_matrix()